In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('agent_df_REFAC.csv')

In [3]:
df = df[df.Step != 0]
df.reset_index(inplace=True)

In [16]:
def aggregate_agent_metrics(df, brand_list=None, channel_list=None):
    """
    Aggregate metrics by step, fully split by brand and channel. Unlike a simple aggregation,
    this function produces brand-specific columns for prices and "units" fields as well,
    rather than single global columns.

    Specifically, it:
      1) Ensures dict-like columns ('Purchased_This_Step', 'Adstock', 'Ad_Channel_Preference')
         are actual Python dicts rather than strings.
      2) Infers brand_list from the first Purchase_This_Step entry, if not provided.
      3) Infers channel_list from the first Ad_Channel_Preference entry, if not provided.
      4) Fills NaNs in Last_Product_Price with Current_Price.
      5) Converts Price_Change to numeric, filling non-numeric entries with 0.0.
      6) Expands per-brand columns for:
           - Purchases (extracted from 'Purchased_This_Step')
           - Adstock (extracted from 'Adstock')
           - Adstock × channel preference (Adstock_{brand}_{channel})
      7) Creates additional brand-specific columns for:
           - Current_Price_{brand}, Last_Product_Price_{brand}, Price_Change_{brand}
             (assigned if the agent's Brand_Choice == brand, else 0)
           - Units_to_Purchase_{brand}, Baseline_Units_{brand}, etc. similarly
      8) Groups by Step, applying:
           - sum for Purchases_{brand}, all per-brand "units" columns
           - mean for Adstock_{brand}, Adstock_{brand}_{channel} (then **rounded up**)
           - mean for brand-level price columns
      9) Returns the aggregated DataFrame with columns for each brand and channel, by step.

    Parameters:
        df (pd.DataFrame): The agent-level simulation output, requiring columns:
            - 'Step'
            - 'Purchased_This_Step' (dict brand->units)
            - 'Adstock' (dict brand->float)
            - 'Ad_Channel_Preference' (dict channel->float)
            - 'Brand_Choice' (string)
            - 'Current_Price', 'Last_Product_Price', 'Price_Change' (floats or possibly strings)
            - 'Units_to_Purchase', 'Baseline_Units', 'Incremental_Promo_Units',
              'Incremental_Ad_Units', 'Decremental_Units'
        brand_list (list, optional): Brand names to process. If None, inferred from the 1st row.
        channel_list (list, optional): Media channel names to process. If None, inferred from 1st row.

    Returns:
        pd.DataFrame: A DataFrame with one row per Step, and brand-specific (and channel-specific) columns.
    """
    import math
    import numpy as np
    import pandas as pd

    # --- 1) Ensure dictionary columns are dict objects ---
    # Convert strings to dict if needed (first row check)
    if isinstance(df["Purchased_This_Step"].iloc[0], str):
        df["Purchased_This_Step"] = df["Purchased_This_Step"].apply(eval)
    if isinstance(df["Adstock"].iloc[0], str):
        df["Adstock"] = df["Adstock"].apply(eval)
    if isinstance(df["Ad_Channel_Preference"].iloc[0], str):
        df["Ad_Channel_Preference"] = df["Ad_Channel_Preference"].apply(eval)

    # --- 2) Infer brand_list if not provided ---
    if brand_list is None:
        first_purchase_dict = df["Purchased_This_Step"].iloc[0]
        brand_list = list(first_purchase_dict.keys())

    # --- 3) Infer channel_list if not provided ---
    if channel_list is None:
        first_channel_dict = df["Ad_Channel_Preference"].iloc[0]
        channel_list = list(first_channel_dict.keys())

    # --- 4) Fill NaNs in Last_Product_Price with Current_Price (if both exist) ---
    if "Last_Product_Price" in df.columns and "Current_Price" in df.columns:
        df["Last_Product_Price"] = df["Last_Product_Price"].fillna(df["Current_Price"])

    # --- 5) Convert Price_Change to numeric, fill non-numeric with 0.0 ---
    if "Price_Change" in df.columns:
        df["Price_Change"] = pd.to_numeric(df["Price_Change"], errors="coerce").fillna(0.0)

    # --- 6) Expand brand-level purchases and adstock columns ---
    for brand in brand_list:
        df[f"Purchases_{brand}"] = df["Purchased_This_Step"].apply(
            lambda p_dict: p_dict.get(brand, 0)
        )
        df[f"Adstock_{brand}"] = df["Adstock"].apply(
            lambda a_dict: a_dict.get(brand, 0)
        )
        # brand × channel adstock
        for channel in channel_list:
            df[f"Adstock_{brand}_{channel}"] = df.apply(
                lambda row: row[f"Adstock_{brand}"]
                             * row["Ad_Channel_Preference"].get(channel, 0.0),
                axis=1
            )

    # --- 7) Create brand-specific columns for single-valued fields ---
    #        We set them to the column's value if brand == row["Brand_Choice"], else 0
    units_cols = [
        "Units_to_Purchase",
        "Baseline_Units",
        "Incremental_Promo_Units",
        "Incremental_Ad_Units",
        "Decremental_Units",
    ]
    price_cols = ["Current_Price", "Last_Product_Price", "Price_Change"]

    # Initialize new columns for each brand
    for brand in brand_list:
        for col in units_cols + price_cols:
            if col in df.columns:  # only if it exists
                df[f"{col}_{brand}"] = 0.0

    # Now fill them in where Brand_Choice == brand
    for brand in brand_list:
        mask = (df["Brand_Choice"] == brand)
        for col in units_cols + price_cols:
            if col in df.columns:
                df.loc[mask, f"{col}_{brand}"] = df.loc[mask, col]

    # --- 8) Build aggregator rules ---
    agg_dict = {}

    # (a) Brand purchases -> sum
    for brand in brand_list:
        agg_dict[f"Purchases_{brand}"] = "sum"

    # (b) Brand & brand-channel adstock -> mean
    for brand in brand_list:
        agg_dict[f"Adstock_{brand}"] = "mean"
        for channel in channel_list:
            agg_dict[f"Adstock_{brand}_{channel}"] = "mean"

    # (c) For brand-specific "units" columns -> sum
    for brand in brand_list:
        for col in units_cols:
            colname = f"{col}_{brand}"
            if colname in df.columns:
                agg_dict[colname] = "sum"

    # (d) For brand-specific price columns -> mean
    for brand in brand_list:
        for col in price_cols:
            colname = f"{col}_{brand}"
            if colname in df.columns:
                agg_dict[colname] = "mean"

    # --- 9) Group by Step and aggregate ---
    aggregated_df = df.groupby("Step").agg(agg_dict).reset_index()

    # --- 10) Round up the adstock columns to nearest int ---
    adstock_cols = [c for c in aggregated_df.columns if c.startswith("Adstock_")]
    for col in adstock_cols:
        aggregated_df[col] = aggregated_df[col].apply(lambda x: math.ceil(x))

    return aggregated_df

In [17]:
aggregate_agent_metrics(df,brand_list=['A','B'], channel_list=['TV','Web'])

,Step,Purchases_A,Purchases_B,Adstock_A,Adstock_A_TV,Adstock_A_Web,Adstock_B,Adstock_B_TV,Adstock_B_Web,Units_to_Purchase_A,...,Baseline_Units_B,Incremental_Promo_Units_B,Incremental_Ad_Units_B,Decremental_Units_B,Current_Price_A,Last_Product_Price_A,Price_Change_A,Current_Price_B,Last_Product_Price_B,Price_Change_B
0,1,8,8,1,1,1,1,1,1,8.0,...,8.0,0.0,0.0,0.0,2.70,2.70,0.0,2.30,2.30,0.0
1,2,15,31,1,1,1,1,1,1,15.0,...,31.0,0.0,0.0,0.0,2.40,2.40,0.0,2.60,2.60,0.0
2,3,29,30,1,1,1,1,1,1,29.0,...,30.0,0.0,0.0,0.0,2.55,2.55,0.0,2.45,2.45,0.0
3,4,40,21,1,1,1,1,1,1,40.0,...,21.0,0.0,0.0,0.0,3.15,3.15,0.0,1.85,1.85,0.0
4,5,61,30,37611,11355,26257,1,1,1,61.0,...,30.0,0.0,0.0,0.0,2.85,2.85,0.0,2.15,2.15,0.0
5,6,47,29,47831,14450,33382,1,1,1,47.0,...,29.0,0.0,0.0,0.0,3.25,3.25,0.0,1.75,1.75,0.0
6,7,41,28,50769,15343,35427,1,1,1,41.0,...,28.0,0.0,0.0,0.0,3.55,3.55,0.0,1.45,1.45,0.0
7,8,50,27,51667,15617,36051,1,1,1,50.0,...,27.0,0.0,0.0,0.0,3.45,3.45,0.0,1.55,1.55,0.0
8,9,39,25,14350,4353,9998,1,1,1,39.0,...,25.0,0.0,0.0,0.0,3.50,3.50,0.0,1.50,1.50,0.0
9,10,41,25,4231,1288,2943,1,1,1,41.0,...,25.0,0.0,0.0,0.0,3.60,3.60,0.0,1.40,1.40,0.0
